In [3]:
#!pip install -U sentence-transformers

In [1]:
from sentence_transformers import SentenceTransformer
from dotenv import load_dotenv
import os
from langchain_community.graphs import Neo4jGraph
import pandas as pd

In [2]:
#Load sentence transformer model ('intfloat/multilingual-e5-large-instruct')
model = SentenceTransformer('intfloat/multilingual-e5-large-instruct')

In [3]:
# Each query must come with a one-sentence instruction that describes the task

def get_detailed_instruct(task_description: str, query: str) -> str:
    return f'Instruct: {task_description}\nQuery: {query}'


task = 'Dado uma query de busca na web, recupere trechos relevantes que respondam à consulta'
queries = [
    get_detailed_instruct(task, 'how much protein should a female eat'),
    get_detailed_instruct(task, '南瓜的家常做法')
]
# No need to add instruction for retrieval documents
documents = [
    "As a general guideline, the CDC's average requirement of protein for women ages 19 to 70 is 46 grams per day. But, as you can see from this chart, you'll need to increase that if you're expecting or training for a marathon. Check out the chart below to see how much protein you should be eating each day.",
    "1.清炒南瓜丝 原料:嫩南瓜半个 调料:葱、盐、白糖、鸡精 做法: 1、南瓜用刀薄薄的削去表面一层皮,用勺子刮去瓤 2、擦成细丝(没有擦菜板就用刀慢慢切成细丝) 3、锅烧热放油,入葱花煸出香味 4、入南瓜丝快速翻炒一分钟左右,放盐、一点白糖和鸡精调味出锅 2.香葱炒南瓜 原料:南瓜1只 调料:香葱、蒜末、橄榄油、盐 做法: 1、将南瓜去皮,切成片 2、油锅8成热后,将蒜末放入爆香 3、爆香后,将南瓜片放入,翻炒 4、在翻炒的同时,可以不时地往锅里加水,但不要太多 5、放入盐,炒匀 6、南瓜差不多软和绵了之后,就可以关火 7、撒入香葱,即可出锅"
]
input_texts = queries + documents

In [4]:
embeddings = model.encode(input_texts, convert_to_tensor=True, normalize_embeddings=True)
scores = (embeddings[:2] @ embeddings[2:].T) * 100
print(scores.tolist())

[[90.71438598632812, 67.90845489501953], [71.38566589355469, 88.65879821777344]]


Connecting to Neo4j

In [5]:
load_dotenv()

# Neo4j variables
NEO4J_URL = os.getenv("NEO4J_URL")
NEO4J_USERNAME =os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")

#Connecting to the graph
graph = Neo4jGraph(
    url=NEO4J_URL,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD
)

Criando embeddings para as teses

In [52]:
#Quering the thesis from graph unsing Cypher
result = graph.query("""
MATCH (thesis:Thesis)
WHERE (thesis.vector) IS NULL
RETURN thesis.uri, thesis.title, thesis.abstract
""")

In [ ]:
for round in range(10):
    print("Round ", round + 1)

    #Quering the thesis from graph unsing Cypher
    result = graph.query("""
    MATCH (thesis:Thesis)
    WHERE (thesis.vector) IS NULL
    RETURN thesis.uri, thesis.title, thesis.abstract
    """)

    ### Transformamos o resultado em um DataFrame e CSV.

    try:
        embedding_csv = pd.read_csv('data/embedding_abstr.csv') 

    except:
        embedding_csv = pd.DataFrame(data={'uri':[], 'embeddings': []})

    batch_size = 50
    n = 0

    texts_list = []
    thesis_uri = []
    embeddings = []

    for thesis in result[:250]:
        n = n + 1   

        texts_list.append(str(thesis['thesis.title']) + '\n   \n' + str(thesis['thesis.abstract']))
        thesis_uri.append(thesis['thesis.uri'])

        if n % batch_size == 0:
            print ("Documents processed: ", n)
            embs_list = model.encode(texts_list, convert_to_tensor=True, normalize_embeddings=True).tolist()

            for i in range(len(thesis_uri)):
                embeddings.append(embs_list[i])

            embedding_csv_batch = pd.DataFrame(data={'uri':thesis_uri, 'embeddings': embeddings})
            embedding_csv = pd.concat([embedding_csv, embedding_csv_batch], ignore_index=True)
            embedding_csv.to_csv('data/embedding_abstr.csv', index=False)

            texts_list = []
            thesis_uri = []
            embeddings = []

    if embeddings != []:
        print ("Documents processed: ", n)
        embs_list = model.encode(texts_list, convert_to_tensor=True, normalize_embeddings=True).tolist()

        for i in range(len(thesis_uri)):
            embeddings.append(embs_list[i])

        embedding_csv_batch = pd.DataFrame(data={'uri':thesis_uri, 'embeddings': embeddings})
        embedding_csv = pd.concat([embedding_csv, embedding_csv_batch], ignore_index=True)
        embedding_csv.to_csv('data/embedding_abstr.csv', index=False)

    # Usando cypher para ler o arquivo CSV
    graph.query("""
        LOAD CSV WITH HEADERS
        FROM 'file:///C:/Users/facordei/OneDrive%20-%20Capgemini/Documents/GitHub/Indigenous-Slavery-KG/data/embedding_abstr.csv' AS row
        MATCH (n:Thesis) Where n.uri = row.uri
        CALL db.create.setNodeVectorProperty(n, 'vector', apoc.convert.fromJsonList(row.embeddings))
        """)
    
    # deletando o arquivo CSV
    
    os.remove('data/embedding_abstr.csv')


Round  1
Documents processed:  100
Documents processed:  200
Documents processed:  300
Documents processed:  400
Documents processed:  500
Documents processed:  600
Documents processed:  700
Documents processed:  800
Documents processed:  900
Documents processed:  1000


In [ ]:
# Usando cypher para ler o arquivo CSV
#graph.query("""
#    LOAD CSV WITH HEADERS
#    FROM 'file:///C:/Users/facordei/OneDrive%20-%20Capgemini/Documents/GitHub/Indigenous-Slavery-KG/data/embedding_abstr.csv' AS row
#    MATCH (n:Thesis) Where n.uri = row.uri
#    CALL db.create.setNodeVectorProperty(n, 'vector', apoc.convert.fromJsonList(row.embeddings))
#    """)

[]

Criando embedding para as páginas (Excerpts)

In [ ]:
#Quering the excerpts from graph unsing Cypher
#result = graph.query("""
#MATCH (excerpt:Excerpt)
#WHERE (excerpt.vector) IS NULL
#RETURN excerpt.uri, excerpt.description
#""")

Criando ídice para os vetores

In [66]:
# Criando um indice de vetores

dimension = len(model.encode(['texto de teste'], convert_to_tensor=True, normalize_embeddings=True)[0])
model.encode(texts_list, convert_to_tensor=True, normalize_embeddings=True)

# Criando uma label "Document" para os nós "Thesis" e "Excerpt"
graph.query("""
    match (n:Thesis|Excerpt) SET n:Document
    """)

# Creating a vector index
graph.query("""
    CREATE VECTOR INDEX vector_search IF NOT EXISTS
    FOR (n:Document)
    ON n.vector
    OPTIONS {indexConfig: {
    `vector.dimensions`: """ + str(dimension) + """,
    `vector.similarity_function`: 'cosine'}}
    """)

[]

In [ ]:
# Criando o indice Full Text

Buscando teses com vetores similares a um determinado texto

In [89]:
#texto que será usado na busca
#query_text = 'Nelson Mandela'
personagem = 'Abdias Nascimento'
query_text = 'Quem trabalhou com ' + personagem + ' ?'
query_text = 'O que é Bandeirologia?'
detailed_query = [get_detailed_instruct(task, query_text)]
print (detailed_query[0])
# Transformando as query em vetor
query_embedding = model.encode(detailed_query, convert_to_tensor=True, normalize_embeddings=True)[0].tolist()

Instruct: Dado uma query de busca na web, recupere trechos relevantes que respondam à consulta
Query: O que é Bandeirologia?


In [90]:
# Buscando no índice de vetores 
result = graph.query("""
    CALL db.index.vector.queryNodes('vector_search', 300, """ + str(query_embedding) + """)
    YIELD node, score
    RETURN score, node.title, node.abstract, node.description
    LIMIT 10
    """)
result

[{'score': 0.9332412481307983,
  'node.title': 'O bandeirismo não é apenas um tema do passado: bandeirantes, entradas e bandeiras na construção historiográfica no Curso de Bandeirologia (1946)',
  'node.abstract': 'Este trabalho dissertativo visa analisar o material de divulgação intitulado Curso de Bandeirologia publicado em 1946. Oriundo de doze conferências públicas relacionadas ao tema das entradas e bandeiras, realizadas no mesmo ano, na cidade de São Paulo, o evento por situar-se em um novo momento político da história do país evidencia uma tentativa por meio de novas formas de divulgação, de repensar a imagem do bandeirante não apenas como um herói, mas como personagem responsável pelos primeiros passos da formação de diferentes estados brasileiros. O material contém os discursos de historiadores de renome nos estudos brasileiros: Afonso de Taunay, Alfredo Ellis Júnior, Sérgio Buarque de Holanda, Virgílio Correa Filho, Joaquim Ribeiro e Afonso Arinos de Melo Franco, letrados rec

In [9]:
from transformers import TFBertModel
from transformers import AutoTokenizer, BertModel
import os
os.environ['CURL_CA_BUNDLE'] = ''
import torch
from dotenv import load_dotenv
from langchain_community.graphs import Neo4jGraph
import json
import pandas as pd
from openai import AzureOpenAI

#from azure.identity import DefaultAzureCredential, get_bearer_token_provider  

Using Azure OpenAI embedding models

In [10]:
#client_small = AzureOpenAI(
#  api_key = os.getenv("OPENAI_API_KEY"),
#  azure_endpoint = os.getenv("EMBEDDING_SMALL_ENDPOINT"),
#  api_version = os.getenv("API_VERSION"))


In [11]:
client_large = AzureOpenAI(
  api_key = os.getenv("OPENAI_API_KEY"),
  azure_endpoint = os.getenv("EMBEDDING_LARGE_ENDPOINT"),
  api_version = os.getenv("API_VERSION"))

In [12]:
# The function receives a sencence and returns the embedding (1D numpy array)
def get_embeddings_openAI(text):
    #model = "text-embedding-3-small"
    #embedding_small = client_small.embeddings.create(input = text, model=model)
    #return embedding_small.data
    model = "text-embedding-3-large"
    embedding_large = client_large.embeddings.create(input = text, model=model)
    return embedding_large.data


Load BERT Model (later we can change to other model)

In [5]:
# Choose the pretrained model
#model_checkpoint = 'bert-base-multilingual-cased' #'bert-base-multilingual-cased'   #'google/bert_uncased_L-2_H-128_A-2' (Tiny BERT)
# Max number of tokens in the sentence
#max_length= 512 #512 #128

# Load tokenizer
#tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, model_max_length= max_length)

# Load the model
#bert_model = TFBertModel.from_pretrained(model_checkpoint, from_pt=True)
#bert_model = BertModel.from_pretrained(model_checkpoint)

In [ ]:
# The function receives a sencence and returns the embedding (1D numpy array)
#def get_embeddings(text):
#  input_ids = torch.tensor(tokenizer.encode(text, add_special_tokens=True, truncation=True)).unsqueeze(0)  # Batch size 1
#  outputs = bert_model(input_ids)
#  last_hidden_states = outputs[0]
#  pooler_output = torch.mean(last_hidden_states, dim=1) 
#  return(pooler_output.detach().numpy()[0].tolist())

Connecting to Neo4j

In [13]:
load_dotenv()

# Neo4j variables
NEO4J_URL = os.getenv("NEO4J_URL")
NEO4J_USERNAME =os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")

#Connecting to the graph
graph = Neo4jGraph(
    url=NEO4J_URL,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD
)

Criando embeddings para as teses

In [7]:
#Quering the thesis from graph unsing Cypher
result = graph.query("""
MATCH (thesis:Thesis)
RETURN thesis.uri, thesis.title, thesis.abstract
""")

In [137]:
### Transformamos o resultado em um DataFrame e CSV.

try:
    embedding_csv = pd.read_csv('data/embedding_openai_1.csv') 
    #with open('data/embeddings_BERT.json', 'r') as fp:
    #with open('data/embeddings_openai.json', 'r') as fp:
    #    dic_embeddings = json.load(fp)

except:
    embedding_csv = pd.DataFrame(data={'uri':[], 'embeddings': []})
    #dic_embeddings = {}

batch_size = 3
n = 0

texts_list = []
thesis_uri = []
embeddings = []

for thesis in result[:12]:
    n = n + 1   

    texts_list.append(str(thesis['thesis.title']) + '\n   \n' + str(thesis['thesis.abstract']))
    thesis_uri.append(thesis['thesis.uri'])

    if n % batch_size == 0:
        print ("Batch number: ", n)
        embs_list = get_embeddings_openAI(texts_list)

        for i in range(len(thesis_uri)):
            #dic_embeddings[thesis_uri[i]] = embs_list[i].embedding
            embeddings.append(embs_list[i].embedding)

        #with open('data/embeddings_BERT.json', 'w') as fp:
        #with open('data/embeddings_openai.json', 'w') as fp:
        #    json.dump(dic_embeddings, fp)

        embedding_csv_batch = pd.DataFrame(data={'uri':thesis_uri, 'embeddings': embeddings})
        embedding_csv = pd.concat([embedding_csv, embedding_csv_batch], ignore_index=True)
        embedding_csv.to_csv('data/embedding_openai_1.csv', index=False)
        #embedding_csv.to_csv('data/embedding_BERT.csv', index=False)

        texts_list = []
        thesis_uri = []
        embeddings = []

if embeddings != []:
    print ("Batch number: ", n)
    embs_list = get_embeddings_openAI(texts_list)

    for i in range(len(thesis_uri)):
        #dic_embeddings[thesis_uri[i]] = embs_list[i].embedding
        embeddings.append(embs_list[i].embedding)

    #with open('data/embeddings_BERT.json', 'w') as fp:
    #with open('data/embeddings_openai.json', 'w') as fp:
    #    json.dump(dic_embeddings, fp)

    embedding_csv_batch = pd.DataFrame(data={'uri':thesis_uri, 'embeddings': embeddings})
    embedding_csv = pd.concat([embedding_csv, embedding_csv_batch], ignore_index=True)
    embedding_csv.to_csv('data/embedding_openai_1.csv', index=False)
    #embedding_csv.to_csv('data/embedding_BERT.csv', index=False)

Batch number:  3
Batch number:  6
Batch number:  9
Batch number:  12


In [ ]:
# Usando cypher para ler o arquivo CSV
graph.query("""
    LOAD CSV WITH HEADERS
    FROM 'file:///C:/Users/facordei/OneDrive%20-%20Capgemini/Documents/GitHub/Indigenous-Slavery-KG/data/embedding_openai.csv' AS row
    MATCH (n:Thesis) Where n.uri = row.uri
    CALL db.create.setNodeVectorProperty(n, 'embedding_openai', apoc.convert.fromJsonList(row.embeddings))
    """)

Criando embedding para as páginas (Excerpts)

In [14]:
#Quering the excerpts from graph unsing Cypher
result = graph.query("""
MATCH (excerpt:Excerpt)
RETURN excerpt.uri, excerpt.description
""")

In [16]:
### Transformamos o resultado em um DataFrame e CSV.

try:
    embedding_csv = pd.read_csv('data/embedding_openai_excerpts.csv') 
    #with open('data/embeddings_BERT.json', 'r') as fp:
    #with open('data/embeddings_openai.json', 'r') as fp:
    #    dic_embeddings = json.load(fp)

except:
    embedding_csv = pd.DataFrame(data={'uri':[], 'embeddings': []})
    #dic_embeddings = {}

batch_size = 100
n = 0

texts_list = []
excerpt_uri = []
embeddings = []

for excerpt in result:
    n = n + 1   

    texts_list.append(str(excerpt['excerpt.description']))
    excerpt_uri.append(excerpt['excerpt.uri'])

    if n % batch_size == 0:
        print ("Batch number: ", n)
        embs_list = get_embeddings_openAI(texts_list)

        for i in range(len(excerpt_uri)):
            #dic_embeddings[excerpt_uri[i]] = embs_list[i].embedding
            embeddings.append(embs_list[i].embedding)

        #with open('data/embeddings_BERT.json', 'w') as fp:
        #with open('data/embeddings_openai.json', 'w') as fp:
        #    json.dump(dic_embeddings, fp)

        embedding_csv_batch = pd.DataFrame(data={'uri':excerpt_uri, 'embeddings': embeddings})
        embedding_csv = pd.concat([embedding_csv, embedding_csv_batch], ignore_index=True)
        embedding_csv.to_csv('data/embedding_openai_excerpts.csv', index=False)
        #embedding_csv.to_csv('data/embedding_BERT.csv', index=False)

        texts_list = []
        excerpt_uri = []
        embeddings = []

if embeddings != []:
    print ("Batch number: ", n)
    embs_list = get_embeddings_openAI(texts_list)

    for i in range(len(excerpt_uri)):
        #dic_embeddings[thesis_uri[i]] = embs_list[i].embedding
        embeddings.append(embs_list[i].embedding)

    #with open('data/embeddings_BERT.json', 'w') as fp:
    #with open('data/embeddings_openai.json', 'w') as fp:
    #    json.dump(dic_embeddings, fp)

    embedding_csv_batch = pd.DataFrame(data={'uri':excerpt_uri, 'embeddings': embeddings})
    embedding_csv = pd.concat([embedding_csv, embedding_csv_batch], ignore_index=True)
    embedding_csv.to_csv('data/eembedding_openai_excerpts.csv', index=False)
    #embedding_csv.to_csv('data/embedding_BERT.csv', index=False)



Batch number:  100
Batch number:  200
Batch number:  300
Batch number:  400
Batch number:  500
Batch number:  600
Batch number:  700
Batch number:  800
Batch number:  900
Batch number:  1000
Batch number:  1100
Batch number:  1200
Batch number:  1300
Batch number:  1400
Batch number:  1500
Batch number:  1600
Batch number:  1700
Batch number:  1800
Batch number:  1900
Batch number:  2000
Batch number:  2100


In [17]:
# Usando cypher para ler o arquivo CSV
graph.query("""
    LOAD CSV WITH HEADERS
    FROM 'file:///C:/Users/facordei/OneDrive%20-%20Capgemini/Documents/GitHub/Indigenous-Slavery-KG/data/embedding_openai_excerpts.csv' AS row
    MATCH (n:Excerpt) Where n.uri = row.uri
    CALL db.create.setNodeVectorProperty(n, 'embedding_openai', apoc.convert.fromJsonList(row.embeddings))
    """)

[]

In [20]:
str(dimension)

'3072'

In [24]:
# Criando um indice de vetores

dimension = len(get_embeddings_openAI('texto de teste')[0].embedding)

# Criando uma label "Document" para os nós "Thesis" e "Excerpt"
graph.query("""
    match (n:Thesis|Excerpt) SET n:Document
    """)

# Creating a vector index
graph.query("""
    CREATE VECTOR INDEX Thesis_Embeddings IF NOT EXISTS
    FOR (n:Document)
    ON n.embedding_openai
    OPTIONS {indexConfig: {
    `vector.dimensions`: """ + str(dimension) + """,
    `vector.similarity_function`: 'cosine'}}
    """)


[]

In [ ]:
# Criand o indice Full Text

graph.query("""
    CREATE FULLTEXT INDEX Thesis_fulltext IF NOT EXISTS FOR (n:Thesis|Excerpt) ON EACH [n.title, n.abstract, n.description]
    """)

[]

Buscando teses com vetores similares a um determinado texto

In [40]:
#texto que será usado na busca
#query_text = 'Nelson Mandela'
personagem = 'Abdias Nascimento'
query_text = 'Quem trabalhou com ' + personagem + ' ?'
# Transformando as query em vetor
#query_embedding = get_embeddings(query_text)
query_embedding = get_embeddings_openAI(query_text)[0].embedding


In [41]:
# Buscando no índice de vetores 
result = graph.query("""
    CALL db.index.vector.queryNodes('Thesis_Embeddings', 5, """ + str(query_embedding) + """)
    YIELD node, score
    RETURN score, node.title, node.abstract, node.description
    """)
result

[{'score': 0.827033281326294,
  'node.title': None,
  'node.abstract': None,
  'node.description': ' à produção literária. Nesse sentido Oliveira \nSilveira, assim como Abdias, era um profundo conhecedor  da estrutura racista da sociedade \nbrasileira, e assim como ele, atuou em muitas frentes para o seu desmonte.  \nAbdias atuou dentro e fora do âmbito partidário, e sua trajetória artística e intelectual \nsempre se deu concomitante à política. Como já referido anteriormente, na década de 1930 ele \n 141 \n \n \n integrou a Frente Negra Brasileira, na de 1950 candidatou -se a vereador (PTB)138 e na de 1960 \na deputado estadual (PTB); após exílio, retomou a atividade partidária nos anos 1980, quando \nexerceu o mandato de deputado federal (PDT) de 1983 -1986 e se reelegeu para o exercício \n1987 -1990, já nos 1990 foi senador da república no período en tre 1997 e 1999; em todas suas \ncandidaturas concorreu pelo estado do Rio de Janeiro. Chegou a fazer “dobradinha ” com Lélia \nGonzal

Buscando usando full text search

In [42]:
result = graph.query("""
CALL db.index.fulltext.queryNodes("Thesis_fulltext", '""" + query_text + """') YIELD node, score
RETURN score, node.title, node.abstract, node.description 
LIMIT 5 
""")
result

[{'score': 17.276535034179688,
  'node.title': 'Quem disse que Exu não monta? Abdias Nascimento, o cavalo do santo no terreiro da história',
  'node.abstract': 'This work addresses the mystical experience between Adbias Nascimento (1914-2011) and his relationship with the orixá Exu in the context of mystical patronage. Using the metaphor “saint’s horse” (“cavalo do santo”) we investigated the vital place that orixá occupied into Abdias’ long-lived and multifaceted trajectory (political, theater, painting, poetry etc.) To think about the mystique (or “mystical biography”) under the African and diasporic approach implied to acess other geographies, logics, rationalities and epistemologies. We start from the assumption that the decolonization of minds’ process requires, within the Afro-Brazilian context, the work of decolonization of mystics. In Brazil, Abdias is an iconic figure with regard to combat racism and simulacrum, the “myth of racial democracy”. But, Abdias’ trajectory was not s